In [47]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error as rmse
import time
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
import csv

sample = pd.read_csv('sample.csv', header = None)
pid = pd.read_csv('train_features.csv').iloc[:, 0:1]
train_features = pd.read_csv('train_features.csv').iloc[:, 1:]
test_features = pd.read_csv('test_features.csv').iloc[:, 1:]
train_labels = pd.read_csv('train_labels.csv').iloc[:, 1:]


In [44]:
def imputation(X_nan):
    num_pazienti = int(X_nan.shape[0]/12)
    X = np.empty((num_pazienti, X_nan.shape[1]))
    for rows in np.arange(0,num_pazienti*12,12):
        X[int(rows/12),:] = np.nanmean(X_nan[int(rows):int(rows+11), :],axis=0)
    inds = np.where(np.isnan(X))
    medie_X = np.nanmean(X, axis=0)
    X[inds] = np.take(medie_X, inds[1])
    return(X)

In [45]:
def single_imputation(X_nan, method):
    num_pazienti = int(X_nan.shape[0]/12)
    X = np.empty((num_pazienti, X_nan.shape[1]))
    for rows in np.arange(0,num_pazienti*12,12):
        X[int(rows/12),:X_nan.shape[1]] = np.nanmean(X_nan[int(rows):int(rows+11), :],axis=0)


    if method=='mean':
        imp_mean = SimpleImputer(missing_values=np.nan, strategy=method)
        X=imp_mean.fit_transform(X)
    elif method=='median':
        imp_median = SimpleImputer(missing_values=np.nan, strategy=method)
        X=imp_median.fit_transform(X)
    elif method=='most_freq':
        imp_mostfreq = SimpleImputer(missing_values=np.nan, strategy=method)
        X=imp_mostfreq.fit_transform(X)

    return X

# Training of Support vector machine

In [46]:
X_train = single_imputation(np.array(train_features), 'mean')
X_test = single_imputation(np.array(test_features), 'mean')
#output_tot = np.array(X_test[:,0:1])
#X_test = X_test[:,1::]
y_training = np.array(train_labels)
y = y_training[:,0:-4]
tot_medical_test = y.shape[1]
print(tot_medical_test)

ScaleData=True
if ScaleData :
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

nvalid = 0.2 # 0 means no validation set (ratio!)
x_train, x_valid, y_train, y_valid = train_test_split(X_train,y,test_size=nvalid,random_state=0)

/var/folders/s4/bm5cfwp97214173jb5bgzcm40000gn/T/ipykernel_2819/3711884852.py:5: RuntimeWarning: Mean of empty slice
  X[int(rows/12),:X_nan.shape[1]] = np.nanmean(X_nan[int(rows):int(rows+11), :],axis=0)
/var/folders/s4/bm5cfwp97214173jb5bgzcm40000gn/T/ipykernel_2819/3711884852.py:5: RuntimeWarning: Mean of empty slice
  X[int(rows/12),:X_nan.shape[1]] = np.nanmean(X_nan[int(rows):int(rows+11), :],axis=0)


11


# First sub task

In [ ]:
med_test_prob = {}
rmse_vector = []
results_1 = []
for medical_test in range(tot_medical_test):
    start = time.time()
    clf = svm.SVC(probability=True, verbose=0)
    print("begin train", medical_test)
    #clf.fit(x_train, y_train[:,medical_test])
    #output = clf.predict_proba(x_valid)
    clf.fit(X_train, y[:,medical_test])
    output = clf.predict_proba(X_test)
    #med_test_prob = {medical_test : output}
    if medical_test != tot_medical_test:
        results_1.append(output[:,1])
    #rootmse = rmse(output[:,1],y_valid[:,medical_test])
    #rmse_vector.append(rootmse)
    #print("rmse for current fit", rootmse)
    print("time for a fitting", time.time()-start)
    if medical_test==tot_medical_test:
        results_2 = output[:,1]
        print(results_2.shape)


begin train 0
time for a fitting 67.93858098983765
begin train 1
time for a fitting 44.46555709838867
begin train 2
time for a fitting 96.07575583457947
begin train 3
time for a fitting 92.88883829116821
begin train 4
time for a fitting 93.3154308795929
begin train 5
time for a fitting 78.5439829826355
begin train 6
time for a fitting 57.81569695472717
begin train 7
time for a fitting 76.86612915992737
begin train 8
time for a fitting 36.294697999954224
begin train 9
time for a fitting 77.64520215988159
begin train 10


In [ ]:
print(np.shape(results_1))

# Third sub task

In [ ]:
def validate_model(best_lambda, sign):
    Y_train=y_training[:,11:16]
    x_train, x_valid, y_train, y_valid = train_test_split(X_train,Y_train,test_size=nvalid,random_state=0)
    regression_class = linear_model.Ridge(alpha=best_lambda, solver='svd')
    regression_class.fit(x_train, y_train[:,sign])
    pred_label = regression_class.predict(x_valid)
    rootmse = rmse(pred_label, y_valid[:,sign])**0.5
    return rootmse

In [ ]:
vital_signs = 4
Y_train=y_training[:,11:16]

pred_label_vec = []
for sign in range(vital_signs):
    lambda_vec = np.array([0.1, 1, 10, 100, 200, 500, 1000])  #### lambda vector
    number_folds = 10  #### Cross-validation folds
    # 1. iteratively split the date in K = 10 folds
    # K-fold cross-validation through sklearn
    kfolds_class = KFold(number_folds, shuffle=True)
    RMSE_4_lambda = []
    # 2. do ridge regression with iteratively different lambdas
    for lbd in lambda_vec:
        RMSE_list = []
        regression_class = linear_model.Ridge(alpha=lbd, solver='svd')
        for train_index, test_index in kfolds_class.split(X_train):
            x_train, x_validation = X_train[train_index], X_train[test_index]
            y_tr, y_val = Y_train[train_index, sign], Y_train[test_index, sign]

            regression_class.fit(x_train, y_tr)
            pred_label = regression_class.predict(x_validation)

            RMSE_list.append(rmse(pred_label, y_val)**0.5)

        RMSE_list = np.array(RMSE_list)
        RMSE_4_lambda.append(np.average(RMSE_list))

    RMSE_4_lambda = np.array(RMSE_4_lambda)

    best_idx = np.argmin(RMSE_4_lambda)
    best_lambda = lambda_vec[best_idx]

    regression_class = linear_model.Ridge(alpha=best_lambda, solver='svd')
    regression_class.fit(X_train, Y_train[:,sign])
    pred_label = regression_class.predict(X_test)

    pred_label_vec.append(pred_label)
    #print("all rmse", RMSE_4_lambda)
    #print("best lambda", best_lambda)
    interative_rmse = validate_model(best_lambda, sign)
    #print("validation error for current model", interative_rmse)
print("pred_label_vec", pred_label_vec)

# Submission

In [ ]:
## Subtask 1
'''results_1 = np.empty((X_test.shape[0],tot_medical_test))
for medical_test in range(0,tot_medical_test):
    output_1 = clf_1[medical_test].predict_proba(X_test[:,1::])
    print('medical_test ',medical_test,'--> ', output_1.shape)
    results_1[:,medical_test] = output_1[:,0]'''

## Subtask 2
#print(output_2.shape)

## Subtask 3
#results_3 = np.array(pred_label_vec).transpose()
#print(results_3.shape)

In [ ]:
results_1 = np.array(results_1).transpose()
results_2 = np.array(results_2)
results_3 = np.array(pred_label_vec).transpose()
print('output_tot: ',output_tot.shape)
print('results_1: ',results_1.shape)
print('results_2: ',results_2.shape)
print('results_3: ',results_3.shape)
output_tot = np.append(output_tot, results_1, axis = 1)
output_tot = np.append(output_tot, results_2, axis = 1)
output_tot = np.append(output_tot, results_3, axis = 1)



header = np.array(sample)[0,:]
header = np.asmatrix(header)
print('output_tot:', output_tot.shape)
#output_tot = np.append(heading, output_tot, axis = 0)
output_tot = output_tot.astype(np.float16)
output_tot = np.concatenate((header, output_tot), axis=0)
df = pd.DataFrame(output_tot)
df.to_csv('new_submission.zip', index=False, float_format='%.3f', compression='zip', header=False)




with open('sample.csv', 'r') as master, open('submission.csv', 'w') as matched:
    cr = csv.reader(master)
    cw = csv.writer(matched)
    #cw.writerow(next(cr))  # copy title
    heading = np.array(next(cr))
print('heading shape : ',heading.shape)
output_tot = np.append(heading.transpose(), output_tot, axis = 0)
print('output_tot:', output_tot)
np.savetxt('submission.csv', output_tot, delimiter=',')